#### Wednesday, April 24, 2024

Trying again on the re-built llama3 conda environment. 

Nice. The notebook trl-tester/SFTTrainer.ipynb was experiencing the same problems with running on the 4090, and setting the environment variables resolved that. So yeah, doing the same thing here to see if it now works.

Nice! This all runs! Great!

#### Saturday, April 20, 2024

mamba activate llama3

https://colab.research.google.com/drive/1eHNWg9gnaXErdAa8_mcvjMupbSS6rDvi?usp=sharing

[Fine-tune Llama 3 with ORPO](https://huggingface.co/blog/mlabonne/orpo-llama-3)

Dammit ... running into errors trying to get this to run, and I have no idea how to fix them ... sigh.

Ugh ... thinking I should try to create a new environment and see if that makes this go away ... yeah, gonna do that.

////////////////////////////////////

OK ... ran 'docker pull huggingface/transformers-pytorch-gpu', did my dockerfile stuff, and am now going to try this from the 'hfpt_Apr20' container ...

Nope! Still problems! And WTF?! ... it's complaining about not being compatible with the 4090!!!

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# Yup, still getting errors complaining about the 4090 ... 
#   Using RTX 4000 series doesn't support faster communication broadband via P2P or IB. 
#   Please set `NCCL_P2P_DISABLE="1"` and `NCCL_IB_DISABLE="1" or use `accelerate launch` which will do this automatically.

import os

os.environ["NCCL_P2P_DISABLE"]="1"
os.environ["NCCL_IB_DISABLE"]="1"

# Fine-tune Llama 3 with ORPO

> 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

You can run this notebook on Google Colab (I use an L4 GPU).

In [1]:
# !pip install -qqq -U transformers datasets accelerate peft trl bitsandbytes wandb --progress-bar off

In [3]:
import gc
import os

import torch
import wandb
from datasets import load_dataset
# from google.colab import userdata
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

In [4]:
import os
from getpass import getpass

# enter your api key
WANDB_API_KEY = getpass("Enter your API key: ")
os.environ["WANDB_API_KEY"] = WANDB_API_KEY

In [ ]:
# wandb token
import os

wb_token = os.environ['WANDB_API_KEY'] 
print(wb_token)

In [5]:
# if we are running this from a docker container, we need to copy the model into the container ... or wait until it downloads ... 
!ls /root/.cache/huggingface/hub/

ls: cannot access '/root/.cache/huggingface/hub/': Permission denied


In [ ]:
# docker cp  /home/rob/Data3/huggingface/transformers/models--meta-llama--Meta-Llama-3-8B 6a31a0645f4c://root/.cache/huggingface/hub/ 

In [6]:
!ls /root/.cache/huggingface/hub/

ls: cannot access '/root/.cache/huggingface/hub/': Permission denied


In [6]:
# Model
base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "OrpoLlama-3-8B"

# Defined in the secrets tab in Google Colab
# wb_token = userdata.get('wandb')
wandb.login(key=wb_token)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: robkayinto. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/rob/.netrc


True

In [7]:
torch.cuda.get_device_capability()[0]

8

In [8]:
# Set torch dtype and attention implementation
# if torch.cuda.get_device_capability()[0] >= 8:
#     !pip install -qqq flash-attn
#     torch_dtype = torch.bfloat16
#     attn_implementation = "flash_attention_2"
# else:
#     torch_dtype = torch.float16
#     attn_implementation = "eager"

torch_dtype = torch.bfloat16
attn_implementation = "flash_attention_2"

In [9]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

In [10]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

In [11]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# Load model ... this fails! ... hmm Now it works!! So let's keep it!
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# Load model ... Nice! This works ... ugh but get an error when we train ...
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=bnb_config,
#     device_map=device,
#     attn_implementation=attn_implementation
# )

# wow even this fails when we run train ... device imbalance ...
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=bnb_config,
#     device_map='cuda:0',
#     attn_implementation=attn_implementation
# )

# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=bnb_config,
#     device_map='cuda',
#     attn_implementation=attn_implementation
# )



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [13]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

In [14]:
dataset_name = "mlabonne/orpo-dpo-mix-40k"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42).select(range(1000)) # Only use 1000 samples for quick demo


In [15]:
def format_chat_template(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

In [16]:
os.cpu_count()

8

In [17]:
dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)
dataset = dataset.train_test_split(test_size=0.01)

In [18]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    beta=0.1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to="wandb",
    output_dir="./results/",
)

In [19]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
)

/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/990 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [20]:
trainer.train()

# 28m 26.4s

  0%|          | 0/123 [00:00<?, ?it/s]

/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 3.9518, 'grad_norm': 67.96273803710938, 'learning_rate': 8e-07, 'rewards/chosen': -0.12671034038066864, 'rewards/rejected': -0.1537468135356903, 'rewards/accuracies': 0.625, 'rewards/margins': 0.02703649178147316, 'logps/rejected': -1.5374683141708374, 'logps/chosen': -1.2671034336090088, 'logits/rejected': -1.9239332675933838, 'logits/chosen': -1.7270550727844238, 'nll_loss': 3.896526336669922, 'log_odds_ratio': -0.5530319809913635, 'log_odds_chosen': 0.41944652795791626, 'epoch': 0.01}
{'loss': 4.2394, 'grad_norm': 81.11616516113281, 'learning_rate': 1.6e-06, 'rewards/chosen': -0.11838454753160477, 'rewards/rejected': -0.1339520514011383, 'rewards/accuracies': 0.625, 'rewards/margins': 0.01556751225143671, 'logps/rejected': -1.3395205736160278, 'logps/chosen': -1.1838454008102417, 'logits/rejected': -1.8769694566726685, 'logits/chosen': -1.816583514213562, 'nll_loss': 4.17724084854126, 'log_odds_ratio': -0.6219723224639893, 'log_odds_chosen': 0.19614256918430328, 'epoch': 0.

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.1478469371795654, 'eval_runtime': 7.274, 'eval_samples_per_second': 1.375, 'eval_steps_per_second': 0.687, 'eval_rewards/chosen': -0.1265636384487152, 'eval_rewards/rejected': -0.12785358726978302, 'eval_rewards/accuracies': 0.5, 'eval_rewards/margins': 0.0012899443972855806, 'eval_logps/rejected': -1.2785357236862183, 'eval_logps/chosen': -1.2656363248825073, 'eval_logits/rejected': -2.0383644104003906, 'eval_logits/chosen': -1.8044925928115845, 'eval_nll_loss': 1.0753557682037354, 'eval_log_odds_ratio': -0.7249106168746948, 'eval_log_odds_chosen': -0.012751561589539051, 'epoch': 0.2}
{'loss': 0.8922, 'grad_norm': 0.9088292717933655, 'learning_rate': 6.8672566371681416e-06, 'rewards/chosen': -0.10239186882972717, 'rewards/rejected': -0.18430739641189575, 'rewards/accuracies': 0.875, 'rewards/margins': 0.08191552013158798, 'logps/rejected': -1.8430739641189575, 'logps/chosen': -1.0239187479019165, 'logits/rejected': -2.200124979019165, 'logits/chosen': -1.64896130561828

...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.0566794872283936, 'eval_runtime': 7.3378, 'eval_samples_per_second': 1.363, 'eval_steps_per_second': 0.681, 'eval_rewards/chosen': -0.11019761860370636, 'eval_rewards/rejected': -0.10561951249837875, 'eval_rewards/accuracies': 0.5, 'eval_rewards/margins': -0.004578106105327606, 'eval_logps/rejected': -1.0561950206756592, 'eval_logps/chosen': -1.1019761562347412, 'eval_logits/rejected': -2.0454564094543457, 'eval_logits/chosen': -1.785556435585022, 'eval_nll_loss': 0.9805904626846313, 'eval_log_odds_ratio': -0.7608903050422668, 'eval_log_odds_chosen': -0.09250213205814362, 'epoch': 0.61}
{'loss': 0.9757, 'grad_norm': 0.8040533065795898, 'learning_rate': 3.3274336283185836e-06, 'rewards/chosen': -0.0969303771853447, 'rewards/rejected': -0.10706248879432678, 'rewards/accuracies': 0.5, 'rewards/margins': 0.010132111608982086, 'logps/rejected': -1.070624828338623, 'logps/chosen': -0.9693037271499634, 'logits/rejected': -1.7168275117874146, 'logits/chosen': -1.204912424087524

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.0404341220855713, 'eval_runtime': 7.3162, 'eval_samples_per_second': 1.367, 'eval_steps_per_second': 0.683, 'eval_rewards/chosen': -0.1068630963563919, 'eval_rewards/rejected': -0.10082907974720001, 'eval_rewards/accuracies': 0.5, 'eval_rewards/margins': -0.006034019403159618, 'eval_logps/rejected': -1.0082906484603882, 'eval_logps/chosen': -1.0686310529708862, 'eval_logits/rejected': -2.0509276390075684, 'eval_logits/chosen': -1.7935816049575806, 'eval_nll_loss': 0.9634195566177368, 'eval_log_odds_ratio': -0.7701447606086731, 'eval_log_odds_chosen': -0.11303961277008057, 'epoch': 0.81}
{'loss': 0.764, 'grad_norm': 0.4990624487400055, 'learning_rate': 1.5575221238938053e-06, 'rewards/chosen': -0.09459416568279266, 'rewards/rejected': -0.10676292330026627, 'rewards/accuracies': 0.5, 'rewards/margins': 0.012168753892183304, 'logps/rejected': -1.0676292181015015, 'logps/chosen': -0.945941686630249, 'logits/rejected': -2.0798444747924805, 'logits/chosen': -1.868561744689941

TrainOutput(global_step=123, training_loss=1.2200393230934454, metrics={'train_runtime': 1706.2977, 'train_samples_per_second': 0.58, 'train_steps_per_second': 0.072, 'train_loss': 1.2200393230934454, 'epoch': 0.99})

In [21]:
trainer.save_model(new_model)

/home/rob/miniforge3/envs/llama3/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [22]:
# Flush memory
del trainer, model
gc.collect()
gc.collect()
torch.cuda.empty_cache()

In [23]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
fp16_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
fp16_model, tokenizer = setup_chat_format(fp16_model, tokenizer)

In [26]:
# Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, new_model)

In [27]:
model = model.merge_and_unload()

In [28]:
# I aborted this because it was going to take a real, real long time to upload, and I really have no reason to upload this model
# model.push_to_hub(new_model, use_temp_dir=False)

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# tokenizer.push_to_hub(new_model, use_temp_dir=False)

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mlabonne/OrpoLlama-3-8B/commit/c6c09a7fed7ada8362c5c68fc2ae4728058ae7b5', commit_message='Upload tokenizer', commit_description='', oid='c6c09a7fed7ada8362c5c68fc2ae4728058ae7b5', pr_url=None, pr_revision=None, pr_num=None)